## RAG Tabanlı AI Agent

In [8]:
# Gerekli kütüphaneler
%pip install -q langchain qdrant-client langchain-community langchain-google-genai sentence-transformers python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [10]:
# Ortam değişkenlerini yükle
from dotenv import load_dotenv
import os

load_dotenv()  # .env dosyasını oku

# API Key'leri al
google_api_key = os.getenv("GOOGLE_API_KEY")
qdrant_api_key = os.getenv("QDRANT_API_KEY")
qdrant_url = os.getenv("QDRANT_URL")

# Kontroller
if not google_api_key:
    raise ValueError("GOOGLE_API_KEY eksik.")
if not qdrant_api_key or not qdrant_url:
    raise ValueError("QDRANT bilgileri eksik.")
print("✅ API Anahtarları yüklendi.")

✅ API Anahtarları yüklendi.


In [11]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url=qdrant_url,
    api_key=qdrant_api_key
)

try:
    collections = qdrant_client.get_collections().collections
    print(f"✅ Qdrant bağlantısı başarılı. Koleksiyonlar: {[c.name for c in collections]}")
except Exception as e:
    print(f"❌ Hata: {e}")

✅ Qdrant bağlantısı başarılı. Koleksiyonlar: ['pdf_agent_collection']


## PDF Yükle ve Böl

In [12]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# PDF dosyasını yükle
pdf_path = "./data/pdf_deneme.pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load()

# Chunk'lara böl
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(pages)

print(f"✅ Toplam chunk sayısı: {len(docs)}")


✅ Toplam chunk sayısı: 124


In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
# Google Generative AI Embeddings modelini başlat
# Embedding modelini başlat
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)

# Chunk’ları vektörlere çevir
from langchain.schema import Document

# Langchain Document nesnesine çevir
docs = [Document(page_content=page.page_content, metadata=page.metadata) for page in pages]

# Bu embedleri Qdrant’a aktaracağız (bir sonraki adım)


In [14]:
#Embedding’leri Qdrant’a Yükle
from langchain.vectorstores import Qdrant as LangchainQdrant

vectorstore = LangchainQdrant.from_documents(
    documents=docs,
    embedding=embedding_model,
    url=qdrant_url,
    api_key=qdrant_api_key,
    collection_name="pdf_agent_collection"
)

print("Embeddings Qdrant'a yüklendi.")


Embeddings Qdrant'a yüklendi.


In [15]:
#soru-cevap aganti oluşturma(RetrievalQA) 
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [16]:
#test
query = "Bu PDF in yazarları kimler?"
result = qa_chain.invoke(query)

print("Cevap:\n", result['result'])

Cevap:
 Bu PDF'in yazarları Çiğdem KAYA ve ark.


In [17]:
%pip install gradio langchain google-generative-ai qdrant-client langchain-qdrant python-dotenv

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement google-generative-ai (from versions: none)
ERROR: No matching distribution found for google-generative-ai


In [19]:
import os
import gradio as gr
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, GoogleGenerativeAI
from langchain_qdrant import QdrantVectorStore
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

load_dotenv()

# Qdrant ayarları
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
QDRANT_COLLECTION = "docs"

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

def pdf_to_text(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    return documents

def chunk_documents(documents, chunk_size=500, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(documents)

def store_embeddings(chunks):

    vectordb = QdrantVectorStore.from_documents(
        chunks,
        embeddings,
        url=QDRANT_URL,
        api_key=QDRANT_API_KEY,
        collection_name=QDRANT_COLLECTION,
        force_recreate=True,
        timeout=100
    )
    return vectordb

def build_qa_chain(vectordb):
    retriever = vectordb.as_retriever()
    llm = GoogleGenerativeAI(
        model="gemini-2.5-flash",
        google_api_key=os.getenv("GOOGLE_API_KEY")
    )
    qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
    return qa

def upload_and_index(pdf_file):
    try:
        documents = pdf_to_text(pdf_file)
        chunks = chunk_documents(documents)
        vectordb = store_embeddings(chunks)
        return "Döküman başarıyla işlendi ve vektör veritabanına kaydedildi."
    except Exception as e:
        return f"Hata oluştu: {str(e)}"

def answer_question(question):
    try:
        from qdrant_client import QdrantClient
        
        client = QdrantClient(
            url=QDRANT_URL,
            api_key=QDRANT_API_KEY,
        )
        
        vectordb = QdrantVectorStore(
            client=client,
            collection_name=QDRANT_COLLECTION,
            embedding=embeddings
        )
        qa = build_qa_chain(vectordb)
        return qa.run(question)
    except Exception as e:
        return f"Hata oluştu: {str(e)}"

with gr.Blocks() as demo:
    gr.Markdown("# Akıllı Doküman Soru-Cevap Sistemi")
    
    with gr.Row():
        # Sol taraf - Doküman yükleme (dar)
        with gr.Column(scale=1):
            gr.Markdown("## 📄 Doküman Yükle")
            pdf_input = gr.File(label="PDF Dosyası Seçin", file_types=[".pdf"])
            upload_btn = gr.Button("📁 Yükle ve İndeksle", variant="primary", size="sm")
            upload_output = gr.Textbox(label="Durum", interactive=False, lines=2)
            upload_btn.click(upload_and_index, inputs=pdf_input, outputs=upload_output)
        
        # Sağ taraf - Soru-cevap (geniş)
        with gr.Column(scale=2):
            gr.Markdown("## 💬 Soru-Cevap Alanı")
            question_input = gr.Textbox(
                label="Sorunuzu buraya yazın", 
                placeholder="Yüklenen döküman hakkında soru sorun...",
                lines=3
            )
            ask_btn = gr.Button("🔍 Cevabı Getir", variant="secondary", size="lg")
            answer_output = gr.Textbox(
                label="Cevap", 
                interactive=False, 
                lines=8,
                placeholder="Cevap burada görünecek..."
            )
            ask_btn.click(answer_question, inputs=question_input, outputs=answer_output)

demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
